# Fine-tuning Leads to Forgetting

This notebook is for GenAI-ML 2025 Homework 8, focusing on the problem of fine-tuning leading to forgetting. The goal is to fine-tune a model using the GSM8K dataset while observing the effects on previously learned knowledge about safeness.  
研究核心为微调引发遗忘这一问题。实验目标是基于 GSM8K 数据集对模型进行微调，同时观察该过程对模型此前习得的安全相关知识所产生的影响。

In [1]:
# Import Packages
import os
import random
import json
import csv
import re
import torch
from transformers import (
    AutoModelForCausalLM, # imports the model for causal language modeling
    AutoTokenizer, # imports the tokenizer for the model
    BitsAndBytesConfig, # imports the configuration for using bitsandbytes
    pipeline # imports the pipeline for text generation
)
from peft import (
    LoraConfig, # imports the configuration for LoRA
    get_peft_model, # imports the function to get the PEFT model
    PeftModel # imports the PEFT model
)
from datasets import Dataset # Imports the Dataset class from the datasets library
from trl import SFTConfig, SFTTrainer # Imports the SFTConfig and SFTTrainer classes from the trl library
from tqdm import tqdm # Imports the tqdm library for progress bars

In [2]:
# Device checking
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

Using device: cuda
PyTorch version: 2.9.1+cu130


In [8]:
# 资源目录
RES_DIR = "res/hw8/"
# 输出目录
OUTPUT_DIR = "output/hw8/"

os.makedirs(RES_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Download Dataset
!curl -L -o {RES_DIR}gsm8k_train.jsonl -k https://www.csie.ntu.edu.tw/~b10902031/gsm8k_train.jsonl
!curl -L -o {RES_DIR}gsm8k_train_self-instruct.jsonl -k https://www.csie.ntu.edu.tw/~b10902031/gsm8k_train_self-instruct.jsonl
!curl -L -o {RES_DIR}gsm8k_test_public.jsonl -k https://www.csie.ntu.edu.tw/~b10902031/gsm8k_test_public.jsonl
!curl -L -o {RES_DIR}gsm8k_test_private.jsonl -k https://www.csie.ntu.edu.tw/~b10902031/gsm8k_test_private.jsonl
!curl -L -o {RES_DIR}ailuminate_test.csv -k https://www.csie.ntu.edu.tw/~b10902031/ailuminate_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
  0     0   0     0   0     0     0     0  --:--:--  0:00:01 --:--:--     0
  0  4068k   0 16166   0     0  6498     0   0:10:41  0:00:02  0:10:39  6500
  0  4068k   0 32550   0     0  9349     0   0:07:25  0:00:03  0:07:22  9350
  0  4068k   0 32550   0     0  7407     0   0:09:22  0:00:04  0:09:18  7407
  1  4068k   1 48934   0     0  8290     0   0:08:22  0:00:05  0:08:17  9060
  1  4068k   1 48934   0     0  7080     0   0:09:48  0:00:06  0:09:42  9030
  1  4068k   1 48934   0     0  6175     0   0:11:14  0:00:07  0:11:07  6027
  1  4068k   1 48934   0     0  5478     0   0:12:40  0:00:08  0:12:32  3006
  1  4068k   1 48934   0     0  4919     0   0:14:06  0:00:09  0:13:57  29

^C


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   0     0   0     0     0     0  --:--:-- --:--:-- --:--:--     0
  0     0   0     0   0     0     0     0  --:--:--  0:00:01 --:--:--     0
 21 74663  21 16169   0     0  8827     0   0:00:08  0:00:01  0:00:07  8830
 21 74663  21 16169   0     0  5318     0   0:00:14  0:00:03  0:00:11  5322
 43 74663  43 32553   0     0  8441     0   0:00:08  0:00:03  0:00:05  8442
 43 74663  43 32553   0     0  6416     0   0:00:11  0:00:05  0:00:06  6419
 65 74663  65 48937   0     0  8395     0   0:00:08  0:00:05  0:00:03 10214
 65 74663  65 48937   0     0  6949     0   0:00:10  0:00:07  0:00:03  6289
 65 74663  65 48937   0     0  6109     0   0:00:12  0:00:08  0:00:04  6593
 87 74663  87 65321   0     0  7066     0   0:00:10  0:00:09  0:00:01  6082
 87 74663  87 65321   0     0  6256     0   0:00:11  0:00:10  0:00:01  6105
100 7

In [3]:
# Ensure reproducibility in training in pytorch and hf transformers
def set_seed(seed):
    random.seed(seed)
    # np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

## LLM Fine-tuning

In [5]:
# Load Model & Tokenizer

sft_model_name = "unsloth/Llama-3.2-1B-Instruct"
sft_bnb_config = BitsAndBytesConfig( # Configuration for using bitsandbytes
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
sft_model = AutoModelForCausalLM.from_pretrained( # Loads the pre-trained model
    pretrained_model_name_or_path=sft_model_name,
    quantization_config=sft_bnb_config,
    dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
sft_tokenizer = AutoTokenizer.from_pretrained( # Loads the tokenizer for the model
    pretrained_model_name_or_path=sft_model_name,
)
sft_tokenizer.model_max_length = 10000
sft_tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Adds a special token for padding
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

peft_model = get_peft_model(sft_model, peft_config).to(dtype=torch.bfloat16)

In [9]:
# Dataset Formatting Functions

def load_jsonlines(file_name: str):
    f = open(file_name, 'r')
    return [json.loads(line) for line in f]

def nshot_chats(nshot_data: list, n: int, question: str, answer: any, mode: str) -> dict: # Function to create n-shot chats
    if mode not in ['train', 'test']:
        raise AssertionError('Undefined Mode!!!')

    chats = []
    # TODO: Use fixed few-shot examples
    for qna in random.sample(nshot_data, n): # Samples n examples from the n-shot data
        chats.append(
            {
                'role': 'user',
                'content': f'Q: {qna["question"]}' # Creates a user message with the question
            }
        )
        chats.append(
            {
                'role': 'assistant',
                'content': f'A: {qna["answer"]}' # Creates an assistant message with the answer
            }
        )

    chats.append(
        {
            'role': 'user',
            'content': f'Q: {question} Let\'s think step by step. At the end, you MUST write the answer as an integer after \'####\'.' # Creates a user message with the question and instructions
        }
    )
    if mode == 'train':
        chats.append(
            {
                'role': 'assistant',
                'content': f'A: {answer}' # Creates an assistant message with the answer
            }
        )

    return chats # Returns the list of chats

In [10]:
# Format GSM8K Data for Fine-tuning

# Filter GSM8K by Length
gsm8k_train = load_jsonlines(os.path.join(RES_DIR, 'gsm8k_train.jsonl')) # Loads the GSM8K training data

formatted_gsm8k = []
# TRAIN_N_SHOT = 1 # TODO: Give model more examples
TRAIN_N_SHOT = 8
for qna in gsm8k_train: # Iterates over the GSM8K training data
    chats = nshot_chats(nshot_data=gsm8k_train, n=TRAIN_N_SHOT, question=qna['question'], answer=qna['answer'], mode='train') # Creates n-shot chats for the current example
    train_sample = sft_tokenizer.apply_chat_template(chats, tokenize=False) # Applies the chat template to the chats
    train_sample = train_sample[train_sample.index("<|eot_id|>") + len("<|eot_id|>"):] # Remove Cutting Knowledge Date in prompt template
    formatted_gsm8k.append( # Appends the formatted example to the list
        {
            'text': train_sample # Adds the text of the example
        }
    )


formatted_gsm8k = Dataset.from_list(formatted_gsm8k) # Creates a dataset from the list of formatted examples

# Sample 1/3 of the longest data. Please do not modify this part.
# Keep the longest 1/3 of `formatted_gsm8k` by letter count
PORTION = 1/3  # change this if needed

def _letters(s):
    s = "" if s is None else (s if isinstance(s, str) else str(s))
    return sum(1 for ch in s if ch.isalpha())

# Choose fields: prefer 'text' if present, else fall back to ('question','answer')
cols = getattr(formatted_gsm8k, "column_names", None) or []
FIELDS = ("text",) if "text" in cols else ("question", "answer")

n = len(formatted_gsm8k)
k = max(1, int(round(n * PORTION)))

# Compute lengths and take top-k indices
lengths = []
for i in range(n):
    ex = formatted_gsm8k[i]  # dict-like
    lengths.append(sum(_letters(ex.get(f, "")) for f in FIELDS))

top_idx = sorted(range(n), key=lambda i: lengths[i], reverse=False)[:k] #modified to shortest 1/3
formatted_gsm8k = formatted_gsm8k.select(top_idx)

print(f"formatted_gsm8k filtered: kept {k}/{n} longest examples using fields={FIELDS}.")

formatted_gsm8k filtered: kept 2491/7473 longest examples using fields=('text',).


In [11]:
# trainer
training_arguments = SFTConfig( # Configuration for the SFT trainer
    seed=42,
    data_seed=42,
    output_dir=os.path.join(OUTPUT_DIR, 'sft'),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=1, # TODO: If you use fixed few-shot examples, increase epoch
    logging_strategy="steps",
    logging_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    lr_scheduler_type='linear',
    # learning_rate=3e-4, # TODO: Decrease learning rate
    learning_rate=3e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    bf16=True,
    group_by_length=True,
    dataset_text_field='text',
    report_to='none',
)
trainer = SFTTrainer( # Creates the SFT trainer
    model=peft_model,
    train_dataset=formatted_gsm8k,
    processing_class=sft_tokenizer,
    args=training_arguments,
)

Adding EOS to train dataset:   0%|          | 0/2491 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2491 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2491 [00:00<?, ? examples/s]

In [12]:
# Fine-tuning
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128256}.


Step,Training Loss
63,1.316100
126,1.088000
189,1.058700
252,1.031000
315,1.022900
378,1.030900
441,1.023500
504,1.037700
567,1.025000


TrainOutput(global_step=623, training_loss=1.0657319241886536, metrics={'train_runtime': 859.6669, 'train_samples_per_second': 2.898, 'train_steps_per_second': 0.725, 'total_flos': 1.4979942610440192e+16, 'train_loss': 1.0657319241886536})

## LLM Inference

In [13]:
# Load Adapter Checkpoint
generator = pipeline( # Creates a text generation pipeline
    'text-generation',
    model=sft_model,
    tokenizer=sft_tokenizer,
    pad_token_id=sft_tokenizer.eos_token_id,
    max_new_tokens=256, # TODO: Increase max_new_tokens for longer output
    # TODO: Use greedy decoding strategy
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
adapter_path = os.path.join(OUTPUT_DIR, "sft/checkpoint-567")
pipeline.model = PeftModel.from_pretrained( # Loads the adapter checkpoint
    sft_model,
    adapter_path,
    torch_dtype=torch.bfloat16,
)
pipeline.model.to(dtype=torch.bfloat16, device="cuda")

Device set to use cuda:0
C:\Users\ROG\AppData\Roaming\Python\Python313\site-packages\peft\tuners\tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [14]:
m = pipeline.model  # or your variable holding the PEFT-wrapped model
print("GPU:", torch.cuda.get_device_name(0), "bf16_supported:", torch.cuda.is_bf16_supported())
print("First param dtype:", next(m.parameters()).dtype)

# Count float32 linears and list suspicious ones
f32_modules = []
for name, mod in m.named_modules():
    if isinstance(mod, torch.nn.Linear):
        if getattr(mod, "weight", None) is not None and mod.weight.dtype == torch.float32:
            f32_modules.append(name)

print(f"# of float32 nn.Linear modules: {len(f32_modules)}")
print("Sample (up to 20):", f32_modules[:20])

# Check embeddings and lm_head explicitly
if hasattr(m, "get_input_embeddings") and m.get_input_embeddings() is not None:
    print("input_embeddings.weight:", m.get_input_embeddings().weight.dtype)
if hasattr(m, "get_output_embeddings") and m.get_output_embeddings() is not None:
    print("output_embeddings(lm_head).weight:", m.get_output_embeddings().weight.dtype)

# Check LoRA params explicitly
lora_f32 = [n for n,p in m.named_parameters() if "lora_" in n and p.dtype == torch.float32]
print("LoRA float32 params (first 20):", lora_f32[:20])

GPU: NVIDIA GeForce RTX 5080 Laptop GPU bf16_supported: True
First param dtype: torch.bfloat16
# of float32 nn.Linear modules: 0
Sample (up to 20): []
input_embeddings.weight: torch.bfloat16
output_embeddings(lm_head).weight: torch.bfloat16
LoRA float32 params (first 20): []


## GSM8K

In [17]:
# GSM8K
def get_response(chats: list): # Function to get the response from the model
    gen_text = generator(chats)[0]  # First return sequence
    return gen_text['generated_text'][-1]['content'] # Returns the content of the last generated text

def extract_ans_from_response(answer: str): # Function to extract the answer from the response
    answer = answer.split('####')[-1].strip() # Splits the answer by '####' and takes the last part

    for remove_char in [',', '$', '%', 'g']: # Removes unwanted characters from the answer
        answer = answer.replace(remove_char, '')

    return answer # Returns the extracted answer

In [20]:
gsm8k_predictions = []
TEST_N_SHOT = 4 # TODO: give model more examples

gsm8k_test_public = load_jsonlines(os.path.join(RES_DIR,'gsm8k_test_public.jsonl')) # Loads the GSM8K public test data
gsm8k_test_public = gsm8k_test_public[0:100] # We use only 100 of the original 13
gsm8k_total = len(gsm8k_test_public) # Gets the total number of examples in the public test data
gsm8k_progress_bar = tqdm(total=gsm8k_total, desc='GSM8K Public Test Data Evaluation', postfix='Current Accuracy = 0.000') # Creates a progress bar for the public test data evaluation

correct = 0

for i, qna in enumerate(gsm8k_test_public): # Iterates over the public test data

    messages = nshot_chats(nshot_data=gsm8k_train, n=TEST_N_SHOT, question=qna['question'], answer=None, mode='test') # Creates n-shot chats for the current example
    response = get_response(messages) # Gets the response from the model

    pred_ans = extract_ans_from_response(response) # Extracts the predicted answer from the response
    true_ans = extract_ans_from_response(qna["answer"]) # Extracts the true answer from the example
    if pred_ans == true_ans: # Checks if the predicted answer is correct
        correct += 1 # Increments the correct count if the prediction is correct
    gsm8k_predictions.append(pred_ans) # Appends the predicted answer to the list of predictions

    gsm8k_progress_bar.set_postfix_str(f'Current Accuracy = {correct/(i+1):.3f}') # Updates the progress bar with the current accuracy
    gsm8k_progress_bar.update() # Updates the progress bar

gsm8k_progress_bar.close() # Closes the progress bar

print(f'GSM8K Public Test Data Evaluation Complete, Total Accuracy: {correct/gsm8k_total:.3f}') # Prints the total accuracy on the public test data

gsm8k_test_private = load_jsonlines(os.path.join(RES_DIR,'gsm8k_test_private.jsonl')) # Loads the GSM8K private test data
gsm8k_test_private = gsm8k_test_private[0:100]
gsm8k_total = len(gsm8k_test_private) # Gets the total number of examples in the private test data
gsm8k_progress_bar = tqdm(total=gsm8k_total, desc='GSM8K Private Test Data Inference') # Creates a progress bar for the private test data evaluation

for i, qna in enumerate(gsm8k_test_private): # Iterates over the private test data

    messages = nshot_chats(nshot_data=gsm8k_train, n=TEST_N_SHOT, question=qna['question'], answer=None, mode='test') # Creates n-shot chats for the current example
    response = get_response(messages) # Gets the response from the model

    pred_ans = extract_ans_from_response(response) # Extracts the predicted answer from the response
    gsm8k_predictions.append(pred_ans) # Appends the predicted answer to the list of predictions

    gsm8k_progress_bar.update() # Updates the progress bar

gsm8k_progress_bar.close() # Closes the progress bar

print(f'GSM8K Private Test Data Inference Complete') # Prints a message indicating that the private test data evaluation is complete

GSM8K Public Test Data Evaluation:  33%|███▎      | 33/100 [02:18<04:31,  4.05s/it, Current Accuracy = 0.242]

AcceleratorError: CUDA error: unknown error
Search for `cudaErrorUnknown' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## AILuminate

In [15]:
def load_csv(file_name: str):
    csvfile = open(file_name)
    rows = csv.DictReader(csvfile)
    questions = []
    for row in rows:
        questions.append(row['prompt_text'])
    return questions

In [18]:
ailuminate_predictions = []

ailuminate_test = load_csv(os.path.join(RES_DIR, 'ailuminate_test.csv')) # Loads the AILuminate test data
ailuminate_public = ailuminate_test[0:40]
ailuminate_private = ailuminate_test[120:160]
ailuminate_test = ailuminate_public + ailuminate_private
ailuminate_total = len(ailuminate_test) # Gets the total number of examples in the AILuminate test data
ailuminate_progress_bar = tqdm(total=ailuminate_total, desc='AILuminate Test Data Evaluation') # Creates a progress bar for the AILuminate test data evaluation

for i, question in enumerate(ailuminate_test): # Iterates over the AILuminate test data

    message = [
        {
            'role': 'user',
            'content': question
        }
    ]
    response = get_response(message) # Gets the response from the model
    ailuminate_predictions.append(response) # Appends the response to the list of predictions

    ailuminate_progress_bar.update() # Updates the progress bar
ailuminate_progress_bar.close() # Closes the progress bar

print(f'AIluminate Test Data Evaluation Complete')

AILuminate Test Data Evaluation:   0%|          | 0/80 [00:16<?, ?it/s]
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
AILuminate Test Data Evaluation: 100%|██████████| 80/80 [16:51<00:00, 12.64s/it]

AIluminate Test Data Evaluation Complete


In [21]:
# Create Submission File
submission_file = os.path.join(OUTPUT_DIR, 'submission.txt')
with open(file=submission_file, mode='w') as f: # Opens the submission file for writing
    print(gsm8k_predictions + ailuminate_predictions, file=f)